In [ ]:
from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.CTR import GCTR

from session_storage import SessionStorage

То, чем мы тут будем заниматься, называется переранжирование -- у нас, фактически, уже есть каким-то образом отранжированная выдача (из серпов сессий), а мы хотим только переупорядочить документы и посмотреть что из этого получился.

Формирование таблички инкапсулировано в SessionStorage, тут мы будем производить лишь качественные действия

In [ ]:
ya_storage = SessionStorage("./data/YandexRelPredChallenge.txt")
vk_storage = SessionStorage("./data/VKVideoClickSessions.txt")
ya_test = ya_storage.get_test_dataset()
vk_test = vk_storage.get_test_dataset()

ya_test.head()


Как будем это оценивать?

Нам не важны сами значения вероятностей для этой задаче, нам важен только порядок -- чтобы документы с ground truth 1 оказались выше, чем документы с 0

Это похоже на AUC. Но нам не подойдет обычный AUC, потому что порядок важен только внутри одной группы (одного запроса)

Для этого воспользуемся [QueryAUC](https://catboost.ai/en/docs/concepts/loss-functions-ranking#QueryAUC) из CatBoost

**Замечание 1:** переранжировать вручную не требуется, т.к. QueryAUC способен сам понять, как упорядочивать документы внутри группы по предсказанным вероятностям (Можно проверить, взяв в качестве метки (1 - click_prob), тогда значение AUC явно даст понять, что что-то не так)

**Замечание 2:** На лекции данный подход рассматривался в контексте наличия асессорских оценок, т.е. была некоторая оценка релевантности $\in \{1, \dots, 5\}$, тут же мы можем не использовать асессорские оценки -- пользователи как бы сами оценивают релевантность документа, отдавая ему свои клики

In [ ]:
from catboost.metrics import QueryAUC

# Получение оценок модели
def apply_model(df, model):
    return df.apply(lambda x: model.predict_relevance(x.query, x.document), axis=1)

# Вычисление метрики
def eval_model(df, model):
    return QueryAUC().eval(
        df["clicked"].values, # настоящие метки -- был ли хоть раз клик в этот документ  
        apply_model(df, model).values, # предсказанная вероятность, мы упорядочивали по ней внутри каждой группы документы
        group_id=df["query"].values # в качестве групп возьмем сами запросы
    )[0]


Посмотрим, какие значения получаются для обученных нами ранее моделей

In [ ]:
def compare_models(df_test, storage):
    dbn_model = SDBN()
    dbn_model.train(storage.get_train_sessions())

    ctr_model = GCTR()
    ctr_model.train(storage.get_train_sessions())

    print(f"DBN AUC\t=\t{eval_model(df_test, dbn_model)}")
    print(f"CTR AUC\t=\t{eval_model(df_test, ctr_model)}")
    
print("VK")
compare_models(vk_test, vk_storage)

print("YA")
compare_models(ya_test, ya_storage)


Не смотря на то, что ранжирование по одному признаку это крайне слабый алгоритм ранжирования, так можно быстро оценить и прикинуть какая из моделей и фичей потенциально способна показать себя лучше при дальнейшем использовании в поиске

**Замечание:** Никто не ограничивает нас использовать только одну самую лучшую модель. Мы вполне себе можем использовать несколько моделей вместе, например те же DBN и CTR. Конечно, мы теряем в таком случае способность быстро прикидывать QueryAUC от ранжирования по одной фиче, однако, добавляя все их в большой датасет, ранжирующая модель может найти довольно интересные и полезные комбинации, которые в итоге повысят общее качество